# Readme

TorchClassifier 封装了 PyTorch 训练模型函数，能通过 class 重写进行高自由度的复用。

# load package

In [1]:
import torch
from torchSimple.models import TorchClassifier

# load dataset

使用的数据集为“长江流域水质等级数据”

In [2]:
import numpy as np
import pandas as pd
import os

X = np.loadtxt('/media/titan2/ed68ee25-5900-4420-b4c5-b77ca64f29e2/WZD/WR/lookback28_feture4_X.csv')
y = np.loadtxt('/media/titan2/ed68ee25-5900-4420-b4c5-b77ca64f29e2/WZD/WR/lookback28_feture4_y.csv')

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

# change the shape to import the resnet18 model.
x_train = torch.tensor(x_train).view(-1, 1, 7, 28)
x_test = torch.tensor(x_test).view(-1, 1, 7, 28)
y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)

# Rewrite the "TorchClassifier" class

这里修改的有四个函数，以及batch size，save folder，若需要更多可前往源码复制粘贴修改.

In [3]:
class ResNet18Classifier(TorchClassifier):
    def __init__(self):
        self.batch_size = 2048
        
        # 可修改保存路径
        self.save_folder = './resnet18_model_weights'
        if not os.path.exists(self.save_folder):
            os.makedirs(self.save_folder)
    
    def set_load_weights(self, path='model_weights_0.746.pt'):
        # 设置要加载的模型权重，如果没有, 则会自动跳过，无需设置为None.
        return os.path.join(self.save_folder, path)
    
    def set_criterion(self):
        return torch.nn.CrossEntropyLoss()
    
    def set_optimizer(self, model):
        return torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-8)
    
    def set_scheduler(self, optimizer):
        return torch.optim.lr_scheduler.StepLR(optimizer, step_size=1)

# use the resnet18 model

使用 torchvision 自带的 resnet18 模型，调整输入输出层的参数. 

In [4]:
from torchvision import models
import torch.nn as nn

model = models.resnet18()
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = nn.Linear(in_features=512, out_features=6, bias=True)

使用 fit 训练模型，需要传 6 个参数.

In [ ]:
resn_clf = ResNet18Classifier()
resn_clf.fit(x_train, y_train, x_test, y_test, model, epochs=5)

Let's use 2 GPUs!
--------------------
EPOCH:1/5


  0%|          | 0/87 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

Traning losses:0.000262478890363127; Accuary:0.822, Precision:0.652, Recall:0.592, F1:0.612
Testing losses:0.0002356240584049374; Accuary:0.849, Precision:0.75, Recall:0.66, F1:0.683
Now best f1 score is: 0.683, save the weights parameters
--------------------
EPOCH:2/5


  0%|          | 0/87 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

Traning losses:0.00018538125732447952; Accuary:0.871, Precision:0.777, Recall:0.695, F1:0.728
Testing losses:0.00018729687144514173; Accuary:0.871, Precision:0.783, Recall:0.704, F1:0.735
Now best f1 score is: 0.735, save the weights parameters
--------------------
EPOCH:3/5


  0%|          | 0/87 [00:00<?, ?it/s]

使用 predict 预测结果，需调用 fit 产生的模型参数（保存在 "resnet18_model_weights"文件夹 里，通过“ls -l -R ./resnet18_model_weights/”查看）

In [ ]:
! ls -l -R ./resnet18_model_weights/

可以看到 fit 时候产生了上述多个文件，格式均为 model_weights_0.xxx.pt，其中0.xxx 为 f1 值，选择最高分数即可.

In [ ]:
prediction = resn_clf.predict(x_test, model, load_weights='./Weights/model_weights_0.737.pt')

调用 class 自带的评估指标 estimate

In [ ]:
acc, precision, recall, f1 = resn_clf.estimate(y_test=y_test.numpy(), y_pred=prediction)
print('acc:{}, precision:{}, recall:{}, f1:{}'.format(acc, precision, recall, f1))

# 调用 utils 绘制混淆矩阵

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

from utils import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test.numpy(),
                              prediction)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure(figsize=(10, 10))
plot_confusion_matrix(cnf_matrix, classes=['Ⅰ', 'Ⅱ', 'Ⅲ', 'Ⅳ', 'Ⅴ', 'Ⅴ-'],normalize=True,)
                      title='./resnet18_model_confusion_matrix.png')
plt.show()

# 可考虑是否需要保存整个模型

目的是保存自己写的模型，这样后续可直接 load 整个模型及其最优 参数.

In [ ]:
# Specify a path
PATH = "./resnet18_model_weights/entire_resnet18_model.pt"
# Save
torch.save(model, PATH)

In [ ]:
# Load
model = torch.load(PATH)

查看模型是否加载成功

In [ ]:
model